# k-means clustering

In this notebook, we will experiment with k-means clustering, looking at cases when it succeeds and when it fails to work.


In [ ]:
import numpy as np
import scipy.linalg
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider

RESCALE_DATA = False # We'll modify this in part (f)


Below is a partial implementation of k-means, initialized with means chosen randomly from the input set (Forgy initialization). Essentially, the algorithm is as follows: `update_means`, then `assign_clusters`, repeated until the cluster assignments stabilize. `update_means` sets the mean of each cluster to be its centroid of each cluster, then `assign_cluster` clusters points based on the mean closest to them.

### Part (a)
**Finish the implementation of `update_means` and `assign_clusters`.**


In [ ]:
def assign_clusters(data, means):
    """
    Takes in a n x d data matrix, and a k x d matrix of the means.
    Returns a length-n vector with the index of the closest mean to each data point.
    """
    n, d = data.shape
    k = means.shape[0]
    assert d == means.shape[1], "Means are of the wrong shape"
    out = np.zeros(n)
    for i, x in enumerate(data):
        # Set out[i] to be the cluster whose mean is closest to point x

        ### start assign_cluster ###

        ### end assign_cluster ###
    return out

def update_means(data, clusters):
    """
    Takes in an n x d data matrix, and a length-n vector of the
    cluster indices of each point.
    Computes the mean of each cluster and returns a k x d matrix of the means.
    """
    n, d = data.shape
    assert len(clusters) == n
    k = len(set(clusters))
    cluster_means = []
    for i in range(k):
        # Set `cluster_mean` to be the mean of all points in cluster `i`
        # (Assume at least one such point exists)

        ### start update_means ###

        ### end update_means ###
        cluster_means.append(cluster_mean)
    return np.array(cluster_means)

def cost(data, clusters, means):
    """
    Computes the sum of the squared distance between each point
    and the mean of its associated cluster
    """
    out = 0
    n, d = data.shape
    k = means.shape[0]
    assert means.shape[1] == d
    assert len(clusters) == n
    for i in range(k):
        out += np.linalg.norm(data[clusters == i] - means[i])
    return out

def k_means_cluster(data, k):
    """
    Takes in an n x d data matrix and parameter `k`.
    Yields the cluster means and cluster assignments after
    each step of running k-means, in a 2-tuple.
    """
    n, d = data.shape
    means = data[np.random.choice(n, k, replace=False)]
    assignments = assign_clusters(data, means)
    while True:
        yield means, assignments
        means = update_means(data, assignments)
        if RESCALE_DATA:
            # This is for part (f), not part (a)
            new_assignments = assign_clusters(*transform_points(data, assignments, means))
        else:
            new_assignments = assign_clusters(data, means)
        if np.all(assignments == new_assignments):
            yield means, assignments
            print("Final cost = {}".format(cost(data, assignments, means)))
            break
        assignments = new_assignments


These are just some utility methods that will prove handy when conducting our experiments.


In [ ]:
def final_k_means_cluster(data, k):
    out = list(k_means_cluster(data, k))
    return out[-1]

def plot_clustering(data, means, assignments):
    k = len(means)
    for j in range(k):
        plt.scatter(*data[assignments == j].T)
    plt.scatter(*means.T, marker="x", s=240, c="black")
    plt.show()

def interact_clustering(data, logger):
    history = list(logger)
    k = history[0][0].shape[0]

    def plotter(i):
        plot_clustering(data, *history[i])

    interact(plotter, i=IntSlider(min=0, max=len(history) - 1, continuous_update=False))

def demo(classes, history=False):
    for c in classes:
        plt.scatter(*c.T)
    plt.show()

    points = np.vstack(classes)

    if history:
        interact_clustering(points, k_means_cluster(points, len(classes)))
    else:
        means, assignments = final_k_means_cluster(points, len(classes))
        plot_clustering(points, means, assignments)


### Part (b)
Now that you've completed your implementation, let's see k-means in action! First, we will generate some points from two isotropic Gaussian distributions, stacked together. Our goal will be for k-means to separate out the points from each distribution.


In [ ]:
def gen_gaussian_points(n, mean, sigma):
    return np.random.normal(mean, sigma, [n, 2])


N = 100

class_a = gen_gaussian_points(N, [-1, 0], [1, 1])
class_b = gen_gaussian_points(N, [1, 0], [1, 1])

points = np.vstack([class_a, class_b])

plt.scatter(*class_a.T)
plt.scatter(*class_b.T)


The above points are reasonably well separated, but there is some overlap between the distributions. Now we will run k-means clustering on this (unlabeled) set of points, to see how well they are separated.

Run the below cell a couple of times and see how the clustering works. **Does the initial choice of means (indicated in green) matter, in this case? What happens if we try to fit 3 or more clusters, or if we vary the spacing between the Gaussian means?**


In [ ]:
interact_clustering(points, k_means_cluster(points, 2))


### start two-gaussians-comments ###

### end two-gaussians-comments ###


### Part (c)
Above, we saw the "ideal" case of k-means, with reasonable well-separated clusters each drawn from isotropic Gaussians. Now we will look at some datasets with non-ideal properties, on which k-means performs poorly for various results.

One problem with k-means is that it can be sensitive to the initial choice of means. Below, we construct a dataset  with three equally spaced point sets of roughly equal size sampled from isotropic Gaussians, as well as a fourth point set slightly removed from the other three of a smaller size.

**Run the below cell a few times. Does k-means always succesfully separate the four classes of points as you'd expect?**


In [ ]:
class_a = gen_gaussian_points(N, [-3, -1], [1, 1])
class_b = gen_gaussian_points(N, [3, -1], [1, 1])
class_c = gen_gaussian_points(N, [0, 3], [1, 1])
class_d = gen_gaussian_points(10, [0, 15], [1, 1])


demo([class_a, class_b, class_c, class_d], history=False) # consider changing this to history=True


You should see that in some cases, k-means fails to separate the four groups into their original four distributions. What does it do instead? Why is this the case? How does this affect the cost of the final clustering? **Comment on your observations.** (Consider passing the `history=True` flag into the call to `demo` above, to see the iterations of k-means as it separates the points.)

### start four-clusters ###

### end four-clusters ###


### Part (d)
We next consider an example with three clusters spaced on the x-axis. You should see that k-means does a reasonable job of separating the clusters, at least visually. Now look at the exact output of the algorithm, In particular, look at the estimated cluster means that it returns.  **How do they compare to the actual means of the true distribution of each cluster? Justify this difference. Will it affect the classification of a new test point?**


In [ ]:
class_a = gen_gaussian_points(N, [-2, 0], [1, 1])
class_b = gen_gaussian_points(N, [0, 0], [1, 1])
class_c = gen_gaussian_points(N, [2, 0], [1, 1])

points = np.vstack([class_a, class_b, class_c])

means, assignments = final_k_means_cluster(points, 3)
plot_clustering(points, means, assignments)

print(means)


### start cluster-means ###

### end cluster-means ###


### Part (e)
Now, we will look at what happens when our Gaussians are no longer isotropic, so they have much greater variance in one dimension versus another. Below, we generate two very well separated clusters, but that have high variance in the y-dimension compared to the x-dimension. **Comment on what happens when we apply k-means to cluster them.**


In [ ]:
# what happens if the Gaussians are not isotropic?
RESCALE_DATA = False

class_a = gen_gaussian_points(N, [-3, 0], [1, 10])
class_b = gen_gaussian_points(N, [3, 0], [1, 10])
demo([class_a, class_b])


**Can you justify what's going on? How might we modify k-means to fix this issue, if we had some prior knowledge about the distribution of our data?**

### start anisotropic-observations ###

### end anisotropic-observations ###


### Part (f)
One such modification is to rescale our coordinate system in the `assign_clusters` step, so the average covariance of our clusters is identity. In other words, our algorithm would be to:
 - `update_means`
 - Compute the average covariance $\Sigma$ of all the clusters (in a manner very similar to LDA)
 - `assign_clusters`, but only after transforming our points such that the cluster covariance is identity.
and repeat until our assignments have stabilized.

We will implement this approach and see how it compares to the naive approach that ignores the cluster covariance at each step.

Algebraically, let $X_i$ be the set of all points in class $i$, with each point in a separate column. Then
$$
    \Sigma = \frac{1}{n} \sum_{i=1}^k (X_i - \mu_i)(X_i - \mu_i)^T,
$$
and we want to find a transformation $T$ such that
$$
    I = \frac{1}{n} \sum_{i=1}^k (TX_i - T\mu_i)(TX_i - T\mu_i)^T.
$$

**Implement `transform_points`, which computes and applies the desired linear transformation to the input points to correct for the cluster covariance**


In [ ]:
def transform_points(data, clusters, means):
    """
    `data` is an n x d matrix containing our input points.
    `clusters` is a length-n vector of the cluster indices of each point.
    `means` is a k x d matrix of the means of each cluster.

    We want to return the transformed data and means after applying the desired transformation.
    *IMPORTANT* Do not modify the input arrays, create a new array for your output.
    """
    n, d = data.shape
    k = means.shape[0]
    assert len(clusters) == n
    assert means.shape[1] == d

    # first, we need to compute the average covariance of each cluster
    sigma = np.zeros((d, d))
    for i in range(k):
        # Computes the cluster covariance of the ith cluster, and add it
        # to the running total in `sigma`
        cluster_points = data[clusters == i]
        demeaned_cluster_points = cluster_points - means[i]
        sigma += demeaned_cluster_points.T @ demeaned_cluster_points

    sigma /= n

    # This transformation should "undo" `sigma` on the data points, so the averaged
    # cluster covariance after the transformation is just I
    transformation = ...

    ### start transformation ###

    ### end transformation ###

    # Remember that each *row* of `data` is a point, not each *column*.
    # Depending on your `transformation` above, you may want to edit the
    # below line.
    return data @ transformation, means @ transformation


Let's see what happens when we run k-means clustering with this new step, running on the same dataset as before.


In [ ]:
RESCALE_DATA = True # include the `transform_points` step in `k_means_cluster`

class_a = gen_gaussian_points(N, [-3, 0], [1, 10])
class_b = gen_gaussian_points(N, [3, 0], [1, 10])
demo([class_a, class_b])


Does it work? Remember from part (c) that k-means can be sensitive to initial conditions, so try running the above cell several times to get a sense of the possible outcomes. **Comment on your observations.**

### start transform-comments ###

### end transform-comments ###


This notion of updating our estimate of the distribution of our data each time step is an important idea that we will explore next week, when we introduce the "expectation-maximization algorithm". That algorithm also runs in a loop, alternating between estimating clusters, and trying to model the distribution of each cluster, so what you have designed above is sort of a "lite" version of that algorithm.
